In [ ]:
import pandas as pd
import json
import os
import argparse as ap

In [ ]:
data = json.load(open('histogram_file.dist_analysis.json'))

df = pd.DataFrame(data["peaks"])
global_maxima = data["global_maxima"]
print(df)
print("-"*80)
print(global_maxima)
print("-"*80)
value = data["mean_freq"]
print(value)

In [ ]:
data = pd.read_csv('output-stats.tsv', sep='\t', header=0, index_col=0)
data
# data_info = data[["invalid_kmers", "non_zero_kmers", "seq_length", "%_non_zero"]]
# data_info
# data_info[["non_zero_kmers"]]

# value = data["non_zero_kmers"].sum()
# print(value)


In [ ]:
print(data.info())

In [ ]:
data = pd.read_csv('test_sequences/sequencing_summary_FAT53867_9a53b23a.txt', sep='\t', index_col=0)

#general data check
print(data.describe())
print("-"*40)


##check read_id duplicates
df1 = len(data["read_id"])
df2 = len(data["read_id"].drop_duplicates())
if df1 == df2:
    print("there really aren't any duplicated")







In [ ]:
data = pd.read_csv('test_sequences/sequencing_summary_FAT53867_9a53b23a.txt', sep='\t', index_col=0)
select_data = data[["read_id", "channel", "start_time", "duration", "sequence_length_template", "mean_qscore_template",
                    "end_reason"]]
select_data.reset_index(drop=True, inplace=True)
select_data

In [170]:
classification = ['signal_positive',
 'data_service_unblock_mux_change',
 'signal_negative',
 'unblock_mux_change']
min_ch = 1
max_ch = 100
min_dur = 1
max_dur = max(select_data.duration)
min_q = 5
max_q = 6
min_len = 500
max_len = 5000

filtered_reads = select_data[(select_data["end_reason"].isin(classification)) & 
                             (select_data.channel.between(min_ch, max_ch)) & 
                             (select_data.duration.between(min_dur, max_dur)) &
                             (select_data.mean_qscore_template.between(min_q, max_q)) & 
                             (select_data.sequence_length_template.between(min_len, max_len))]


filtered_reads[["read_id"]]
# filtered_reads[["read_ids"]].to_csv("filtered_reads.csv", index=False)

,read_id
2434,36b8f142-863a-4857-815e-4912bf77f54f
6909,b6df0b84-e5dc-4316-8ed7-dc4166d6b58c
7202,92f8ce66-cb09-46c9-b040-46222e74816d
8593,2ec80c2d-be2d-4e41-afec-1798af1b1a68
14010,1dfe9a84-443b-4428-8984-18530f2f259c
...,...
338351,4005dd9b-fb80-4c96-96f2-22c45ea5b710
338835,87bc71ec-3138-4707-b783-f953afd246dd
342225,626710a8-35aa-48b0-ba88-96993ad304cd
343119,12b4052f-19d8-4845-8321-60491168d8ef


In [ ]:
def check_ont_summary_file(file_path):
    if not os.path.isfile(file_path):
        print("Error: file not found")
        return False

    with open(file_path, "r") as f:
        lines = f.readlines()

    if not lines or len(lines) < 2:
        print("Error: file is empty or has fewer than 2 lines")
        return False
    
    for line in lines[1:]:
        fields = line.strip().split("\t")
        if len(fields) != 41:
            print(f"Error: malformed line: {line.strip()}")
            return False
        try:
            int(fields[4])
        except ValueError:
            print(f"Error: invalid channel designation: {fields[4]}")
            return False
    
    return True

check_ont_summary_file("test_sequences/sequencing_summary_FAT53867_9a53b23a.txt")

In [ ]:
classes = {"stop_receiving":["signal_positive"], "unblocked":["data_service_unblock_mux_change"],
               "no_decision":["signal_negative", "unblock_mux_change"]}

classification = "no_decision"

new_classification = classes[classification]
pp = list(classes.values())
ppp = []
for x in pp:
    ppp.extend(x)
ppp

In [ ]:
with open("test_sequences/Test_br1_sal_lam_enriched.fastq", "r") as f:
    first_line = f.readline().strip()
    second_line = f.readline().strip()
    third_line = f.readline().strip()
    fourth_line = f.readline().strip()

if len(second_line) == len(fourth_line):
    print("yeahh buddy, light weight baby")

In [177]:
parser = ap.ArgumentParser(prog="sequenoscope", usage="%(prog)s <command> <argument>", description="%(prog)s version 1.0: a tool for analyzing and processing sequencing data.",
                           formatter_class= ap.RawTextHelpFormatter)

parser._positionals.title = "Commands"
# parser._positionals
parser._optionals.title = "Help"
# parser.add_argument('--foo', action='store_true', help='foo help')
subparsers = parser.add_subparsers()

analyze_parser = subparsers.add_parser('analyze', help='map reads to a target and produce a report with sequencing statistics')
plot_parser = subparsers.add_parser('plot', help='generate plots based on fastq or kmer hash files')


filter_parser = subparsers.add_parser('filter_ONT', help='filter reads from a fastq file based on a sequencing summary file')
filter_parser.usage = "sequenoscope filter_ONT --input_fastq <file.fq> --input_summary <seq_summary.txt> -o <out.fastq> [options]"
filter_parser.add_argument("--input_fastq", metavar="", required=True, nargs="+", help="[REQUIRED] Path to adaptive sequencing fastq files to process.")
filter_parser.add_argument("--input_summary", metavar="", required=True, help="[REQUIRED] Path to ONT sequencing summary file.")
filter_parser.add_argument("-o", "--output", metavar="", required=True, help="[REQUIRED] Output directory/file designation")
filter_parser.add_argument("-cls", "--classification", metavar="", choices=['unblocked', 'stop_receiving', 'no_decision'], help="a designation of the adaptive-sampling sequencing decision classification ['unblocked', 'stop_receiving', or 'no_decision']")
filter_parser.add_argument("-min_ch", "--minimum_channel", default=1, metavar="", type=int, help="a designation of the minimum channel/pore number for filtering reads")
filter_parser.add_argument("-max_ch", "--maximum_channel", default=512, metavar="", type=int, help="a designation of the maximum channel/pore number for filtering reads")
filter_parser.add_argument("-min_dur", "--minimum_duration", metavar="", type=float, help="a designation of the minimum duration of the sequencing run in SECONDS for filtering reads")
filter_parser.add_argument("-max_dur", "--maximum_duration", metavar="", type=float, help="a designation of the maximum duration of the sequencing run in SECONDS for filtering reads")
filter_parser.add_argument("-min_start", "--minimum_start_time", metavar="", type=float, help="a designation of the minimum start time of the sequencing run in SECONDS for filtering reads")
filter_parser.add_argument("-max_start", "--maximum_start_time", metavar="", type=float, help="a designation of the maximum start time of the sequencing run in SECONDS for filtering reads")
filter_parser.add_argument("-min_q", "--minimum_q_score", metavar="", type=int, help="a designation of the minimum q score for filtering reads")
filter_parser.add_argument("-max_q", "--maximum_q_score", metavar="", type=int, help="a designation of the maximum q score for filtering reads")
filter_parser.add_argument("-min_len", "--minimum_length", metavar="", type=int, help="a designation of the minimum read length for filtering reads")
filter_parser.add_argument("-max_len", "--maximum_length", metavar="", type=int, help="a designation of the maximum read length for filtering reads")
# parser.print_help()
args = parser.parse_args(["filter_ONT", "--input_fastq", "<file.fq>", "--input_summary", "<seq_summary.txt>", "-o", "<out.fastq>", "-cls", "no_decision"])
# filter_parser.print_help()

clas = args.classification
print(clas)

no_decision
